# POWGEN Reduction

A simple reduction workflow for the [POWGEN](https://sns.gov/powgen) beamline.

In [ ]:
import scipp as sc
import scippneutron as scn
from ess import diffraction
from ess import powgen
import ess
import numpy as np

In [ ]:
logger = ess.logging.configure_workflow('powgen_reduction',
                                        filename='powgen.log')

## Load Data

Load the sample data.

In [ ]:
sample = scn.load(ess.powgen.data.sample_file(),
                  advanced_geometry=True,
                  load_pulse_times=False,
                  mantid_args={'LoadMonitors': True})

In [ ]:
sample

Normalize by proton charge

In [ ]:
sample /= sample.attrs['gd_prtn_chrg'].values

## Calibration

Load calibration data.
We need to specify an instrument definition that Mantid udnerstands.

In [ ]:
cal = diffraction.load_calibration(
    ess.powgen.data.calibration_file(),
    instrument_filename='POWGEN_Definition_2011-02-25.xml')

The calibration is loaded with a 'detector' dimension.
Compute the corresponding spectrum indices using the detector info.

In [ ]:
cal = powgen.beamline.map_detector_to_spectrum(cal, detector_info=sample.coords['detector_info'].value)

In [ ]:
cal

In [ ]:
sample_dspacing = diffraction.to_dspacing_with_calibration(sample, calibration=cal)

## Vanadium Correction

Vanadium

In [ ]:
vana = diffraction.load_vanadium(powgen.data.vanadium_file(), powgen.data.empty_instrument_file())

In [ ]:
vana_dspacing = diffraction.to_dspacing_with_calibration(vana, calibration=cal)

In [ ]:
vana_dspacing

We need to histogram the vanadium events in order to normalize our data.
For consistency, we use these bin edges for both vanadium and later the sample data.

In [ ]:
d = vana_dspacing.coords['dspacing']
dspacing_edges = sc.linspace('dspacing',
                             d.min().value,
                             d.max().value,
                             200,
                             unit=d.unit)

### Combine all Spectra

First, compute a corrected d-spacing distribution for all spectra at once.

In [ ]:
normalized = diffraction.normalize_by_vanadium(sample_dspacing.bins.concat('spectrum'),
                                          vanadium=vana_dspacing.bins.concat('spectrum'),
                                          edges=dspacing_edges)

In [ ]:
normalized.plot()

### Focus into $2\theta$ Bins

For better resolution, focus the sample and vanadium data into a number of bins in the scattering angle $2\theta$.
And normalize afterwards.

In [ ]:
two_theta = sc.linspace(dim='two_theta', unit='rad', start=0.0, stop=np.pi, num=16)
sample_focussed = diffraction.focus_by_two_theta(sample_dspacing, edges=two_theta)
vana_focussed = diffraction.focus_by_two_theta(vana_dspacing, edges=two_theta)

In [ ]:
normalized = diffraction.normalize_by_vanadium(sample_focussed, vanadium=vana_focussed, edges=dspacing_edges)

In [ ]:
normalized.plot()

We can also inspect d-spacing for specific ranges of scattering angles.

In [ ]:
angle = normalized.coords['two_theta'].values
angle = 0.5 * (angle[1:] + angle[:-1])
results = {
    f'{round(angle[group], 3)} rad': normalized['dspacing', :100]['two_theta', group]
    for group in range(2, 6)
}
sc.plot(results)